In [1]:
import numpy as np
import pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display
import timeit
from datetime import timedelta
from pathlib import Path
import datetime as dt 
import matplotlib.pyplot as plt
from collections import namedtuple
from scipy.io import loadmat

mat = loadmat('spamData.mat')  # load mat-file
# mat = loadmat('../input/spamdata/spamData.mat')
# sorted(mat.keys())
x_test = mat['Xtest']  # variable in mat file
x_train = mat['Xtrain']  # variable in mat file
y_test = mat['ytest']  # variable in mat file
y_train = mat['ytrain']  # variable in mat file

In [2]:
# data preprocessing: binarization and log transformation
from sklearn import preprocessing

def logTrans(data):
    return np.log(data + 0.1)

xtrain = preprocessing.binarize(x_train)
xtest = preprocessing.binarize(x_test)
ytrain = y_train.flatten()
ytest = y_test.flatten()
xtest_log = logTrans(x_test)
xtrain_log = logTrans(x_train)

In [3]:
# let N1 be # of spam emails, N be total # of emails
N1train = 0
Ntrain = 0
for row in y_train:
#     print (row)
    if row == 1:
        N1train += 1
    Ntrain += 1
N0train = Ntrain - N1train
print (N1train, Ntrain, N1train/Ntrain)
MLE = np.log(N1train / Ntrain)
MLE_neg = np.log(1-N1train/Ntrain)

# # feature likelihood from training data
# feature_likelihood = np.mean(xtrain[ytrain == 1], axis = 0)
# print (feature_likelihood)

1240 3065 0.40456769983686786


In [6]:
def sum_x_train(spammed):
    return np.sum(xtrain[ytrain == spammed], axis = 0).astype('uint16')
def sum_y_train(spammed):
    return len(ytrain[ytrain == spammed])
def posterior_predictive_distribution (spammed, alpha):
    return ((sum_x_train(spammed)+alpha)/(sum_y_train(spammed)+2*alpha))
def classify(email, alpha):
    feature_count = len(email)
    
    param_spam = 0
    param_non_spam = 0
    
    for i in np.arange(feature_count):
        temp_post_dist_0 = posterior_predictive_distribution(0, alpha)[i]
        temp_post_dist_1 = posterior_predictive_distribution(1, alpha)[i]
        param_spam += (email[i] == 1) * np.log(temp_post_dist_1) + \
        (email[i] == 0) * np.log(1-temp_post_dist_1)
        param_non_spam += (email[i] == 1) * np.log(temp_post_dist_0) + \
        (email[i] == 0) * np.log(1-temp_post_dist_0)
        
    prob_spam = MLE + param_spam
    prob_non_spam = MLE_neg + param_non_spam

    return (prob_spam > prob_non_spam).astype('uint8')
def calc_error_rate_test(alpha):
    error_count = 0
    for count, email in enumerate(xtest):
        if (classify(email, alpha) != ytest[count]):
            error_count += 1
    return error_count/len(xtest)
def calc_error_rate_train(alpha):
    error_count = 0
    for count, email in enumerate(xtrain):
        if (classify(email, alpha) != ytrain[count]):
            error_count += 1
    return error_count/len(xtrain)

In [7]:
# compute error rate for test data with different alpha values
alphas = np.arange(0,100.5,0.5)
error_rate_test = []
error_rate_train = []

for alpha in alphas:
    error_rate_test.append(calc_error_rate_test(alpha))
    print ("alpha = ", alpha, "computed")
print (error_rate_test)

for alpha in alphas:
    error_rate_train.append(calc_error_rate_train(alpha))
    print ("alpha = ", alpha, "computed")
print (error_rate_train)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in multiply


alpha =  0.0 computed
alpha =  0.5 computed
alpha =  1.0 computed
alpha =  1.5 computed
alpha =  2.0 computed
alpha =  2.5 computed
alpha =  3.0 computed
alpha =  3.5 computed
alpha =  4.0 computed
alpha =  4.5 computed
alpha =  5.0 computed
alpha =  5.5 computed
alpha =  6.0 computed
alpha =  6.5 computed
alpha =  7.0 computed
alpha =  7.5 computed
alpha =  8.0 computed
alpha =  8.5 computed
alpha =  9.0 computed
alpha =  9.5 computed
alpha =  10.0 computed
alpha =  10.5 computed
alpha =  11.0 computed
alpha =  11.5 computed
alpha =  12.0 computed
alpha =  12.5 computed
alpha =  13.0 computed
alpha =  13.5 computed
alpha =  14.0 computed
alpha =  14.5 computed
alpha =  15.0 computed
alpha =  15.5 computed
alpha =  16.0 computed
alpha =  16.5 computed
alpha =  17.0 computed
alpha =  17.5 computed
alpha =  18.0 computed
alpha =  18.5 computed
alpha =  19.0 computed
alpha =  19.5 computed
alpha =  20.0 computed
alpha =  20.5 computed
alpha =  21.0 computed
alpha =  21.5 computed
alpha = 

KeyboardInterrupt: 

In [8]:
# Q2 Gaussian Naive Bayes

# compute mean and standard deviation for ML estimation
xtrain_log_mean_spam = np.mean(xtrain_log[ytrain == 1], axis = 0)
xtrain_log_std_spam = np.std(xtrain_log[ytrain == 1], axis = 0)
xtrain_log_mean_nonspam = np.mean(xtrain_log[ytrain == 0], axis = 0)
xtrain_log_std_nonspam = np.std(xtrain_log[ytrain == 0], axis = 0)

In [9]:
def classify_gaussian(email):
    feature_count = len(email)
    
    param_spam = 0
    param_non_spam = 0
    
    for i in np.arange(feature_count):
        param_spam += -np.log(xtrain_log_std_spam[i]) - ((email[i] - xtrain_log_mean_spam[i])/(xtrain_log_std_spam[i])) ** 2 / 2
        param_non_spam += -np.log(xtrain_log_std_nonspam[i]) - ((email[i] - xtrain_log_mean_nonspam[i])/(xtrain_log_std_nonspam[i])) ** 2 / 2
        
    prob_spam = MLE + param_spam
    prob_non_spam = MLE_neg + param_non_spam
    
    return (prob_spam > prob_non_spam).astype('uint8')

def error_rate_gaussian(data, result):
    error_count = 0
    
    for count, email in enumerate(data):
        if (classify_gaussian(email) != result[count]):
            error_count += 1
    return (error_count / len(data))

In [14]:
error_rate_gaussian_train = error_rate_gaussian(xtrain_log, ytrain)
error_rate_gaussian_test = error_rate_gaussian(xtest_log, ytest)
print (error_rate_gaussian_train)
print (error_rate_gaussian_test)

0.16802610114192496
0.16341145833333334


In [12]:
# Q3 Logistic Regression
lambdas = np.hstack((np.arange(1,10,1), np.arange(10,105,5)))
lambdas

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  15,  20,  25,
        30,  35,  40,  45,  50,  55,  60,  65,  70,  75,  80,  85,  90,
        95, 100])

In [13]:
def sigmoid(x):
    return 1/(1+exp(-x))
